## Fitting drug response curves with sigmoid function

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import os

from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
_FOLDER = "data/"
_FOLDER_2 = "datasets/"
_FOLDER_3 = "fitted_datasets/"

## Bounds for fitting coeficents

In [2]:
def sigmoid_Wang(x, p, s):
    """ Sigmoid function from Dennis Wang's paper:
    x - dosage [0, 1],
    p - position,        default=0.4 p=0 no upper plateu, p=1 no lower plateu bounds [0, 1]
    s - shape parameter, default=0.1 s<0= normal sigmoid, s>1=straight line, more s, more horisontal line, 
                         general bounds s>0, but for good drug responses [0,1]
    parameters_bound ((0,0], [1,1)) or (0,1)
    """
    return ( 1.0 / (1.0 + np.exp((x-p)/s)) )

def fsigmoid(x, p, k):
    """ Comparing with Dennis Wang's sigmoid:
    x = x  - dosage [0, 1]
    p - position [0,1],           default = 0.4 bounds = [0,1]
    k = -1/s (s -shape parameter) default = -10 k=0 straight line, k<0 sigmoid around k=-10
    parameters_bound ((0,-100],[1,-1))
    """
    return ( 1.0 / (1.0 + np.exp(-k*(x-p))) )


def sigmoid_4_param(x, x0, L, k, d):
    """ Comparing with Dennis Wang's sigmoid:
    x0 -  p - position, correlation with IC50 or EC50
        bounds [0, 1]
    L = 1 in Dennis Wang's sigmoid, protect from devision by zero if x is too small 
        L<1 inverted sigmoid, l=100 - lower upper and lower boundso sigmpoid on y axis (y= [0.1, 0.11])
        bounds [0.8, 10]
    k = -1/s (s -shape parameter)  default = -10 k=0 straight line, k<0 sigmoid around k=-10
        bounds [1, -100]
    d - determines the vertical position of the sigmoid - shift on y axis - better fitting then Dennis Wang's sigmoid
         bounds [0, 0.9]
    parameters_bound ((0, 0.8, -100, 0), (1, 10, 1, 0.9))
    """
    return ( 1/ (L + np.exp(-k*(x-x0))) + d)


def sigmoid_3_param(x, x0, k, d):
    """ Comparing with Dennis Wang's sigmoid:
    x0 -  p - position, correlation with IC50 or EC50
        bounds [0, 1]
    k = -1/s (s -shape parameter)  default = -10 k=0 straight line, k<0 sigmoid around k=-10
        bounds [1, -100]
    d - determines the vertical position of the sigmoid - shift on y axis - better fitting then Dennis Wang's sigmoid
         bounds [0, 0.9]
    parameters_bound ((0, -100, 0], [1, 1, 0.9))"""
    return ( 1/ (1 + np.exp(-k*(x-x0))) + d )


def ll4(x, e, c, b, d):
    """ https://gist.github.com/yannabraham/5f210fed773785d8b638
    This function is basically a copy of the LL.4 function from the R drc package:
     np.exp( b* (np.log(x)-np.log(e)) -  np.exp((x-p)/s in Dennis Wang's sigmoid
     np.log(x)-np.log(e) == x-p in Dennis Wang's sigmoid
     e - EC50 
         bounds [0.1, 1], default 0.4
     c-d - difference between max and min responses
     c - max response
         bounds [0,1], default 1
     b- hill slope = 1/s - shape parameter
         bounds [0, 50], default 5
     d - min response - determines the vertical position of the graph
         bounds [0,1]
     parameters_bound ((0.1,0,0,0],[1, 1, 50, 1))
   
     """
    return ( (c-d)/(1 + np.exp( b*(np.log(x)-np.log(e) ))) + d)


def ll4_R(x, e, c, b, d):
    """ LL.4 function from R
    https://www.rdocumentation.org/packages/drc/versions/2.5-12/topics/LL.4
    e - EC50 - np.log(concentration) np.log(0.0001)=-10, np.log(1)=0
         bounds [-10, 0], default -5
     c-d - difference between max and min responses
     c - max response
         bounds [0,1], default 1
     b - hill slope = 1/s - shape parameter
         bounds [0, 50], default 5
     d - min response - determines the vertical position of the graph
         bounds [0,1]
     parameters_bound ((-10,0,0,0],[0, 1, 50, 1))

    """
    return ( (c-d)/(1+np.exp(b*np.log(x)- e)) + d)


def logistic4(x, A, B, C, d):
    """ https://people.duke.edu/~ccc14/pcfb/analysis.html
    4PL logistic equation
    Dennis Wang's sigmoid: 1.0 / (1.0 + np.exp((x-p)/s)
    (A - d) = 1 in Dennis Wang's sigmoid:
          bounds [0, 1.5] default A = 1
    (x/C)**B  - corresponds to np.exp((x-p)/s
    B - determines shape/ 0.1 half-sigmoid, 5 -good sigmoid
        bounds [0.1, 50], default B = 5
    C - deterimnes position of the centre of sigmoid
        bounds [0, 1], default C = 0.4
    
    d - determines the vertical position of the sigmoid - shift on y axis 
         bounds [0, 0.9], default d = 0
         
    parameters_bound ((0, 0.1, 0, 0], [1.5, 50, 1, 0.9))
    """
    return ( (A-d)/(1.0+((x/C)**B)) + d )


def logLogistR(x, EC50, HS, E_inf):
    """Python analog for PharmacoGx/R/LogLogisticRegression.R
    https://github.com/bhklab/PharmacoGx/blob/master/R/LogLogisticRegression.R
    E = E_inf + (1 - E_inf)/(1 + (x/EC50)^HS)
    Dennis Wang's sigmoid: 1.0 / (1.0 + np.exp((x-p)/s)
    
    (A - d) = 1 in Dennis Wang's sigmoid:
    (np.log10(x)/EC50)**HS  - (in logistic4 (x/C)**B) corresponds to np.exp((x-p)/s 
    EC50 - concentration, determines shape of a sigmoid and its center
        bounds [0, 1], default 0.4
    HS - determines shape:
        bounds [-30, -2] default -4 only even values work
    E_inf - determines the vertical position of the graph /coefficient d, min response in other functions
        bounds [0, 1]
    parameters_bound ((0,-30,0], [1,-2,1))
    """
    return ((1-E_inf)/(1+(np.log10(x)/EC50)**HS) + E_inf)

In [3]:
def FitCurve(fitting_function, x, y, parameters_guess=None, bounds = (-100, 100), to_plot = False):
#     from scipy.optimize import curve_fit

    if parameters_guess:
        parameters, p_covariance = curve_fit(fitting_function, x, y, parameters_guess, bounds= bounds)
    else: 
        parameters, p_covariance = curve_fit(fitting_function, x, y, bounds= bounds)
    x2 = np.linspace(0, 1, 10)
    y_fit = fitting_function(x, *parameters)
    r2 = r2_score(y, y_fit)

    if to_plot:
        print("Fitting parameters:", *parameters)
        plt.scatter(x, y)
        x2 = np.linspace(0, 1, 10)
        y2 = fitting_function(x2, *parameters)
        plt.plot(x2, y2, "blue", label = "R^2= %0.5f"%r2)   
        plt.title('Least-squares fit')
        plt.legend();
    return r2, parameters


def FittingColumn(df, indexes, x_columns, y_columns, fitting_function, parameters_guess=None, default_param = False):
    """
    intial parameter guess [max(y), np.median(x), 1, min(y)]
    potentially they can be different for each data row, but as soon as we have scaled and filtered data
    we can use by default [1.0, 0.4, 1.0, .0] 
    """
    
    r2_scores = np.zeros(len(indexes))
    X = df.loc[indexes, x_columns].values.astype(np.float32)
    Y = df.loc[indexes, y_columns].values.astype(np.float32)
    fitting_parameters = [None]*len(indexes)
    
    
    # parameters_guess= [np.median(x), 1, max(y), min(y)]
    default_param_model = {"sigmoid_Wang": {"initial_guess": [0.4, 0.1], 
                                             "bounds": (0, 1)},
                           "fsigmoid" : {"initial_guess": [0.4, -10], 
                                          "bounds": ((0,-100), (1,-1))},
                           "sigmoid_4_param": {"initial_guess": [0.4, 1.0, -10, .0], 
                                                "bounds": ((0, 0.8, -100, 0), (1, 10, 1, 0.9))},
                           "sigmoid_3_param": {"initial_guess": [0.4, -10.0, .0],  
                                               "bounds": ((0, -100, 0), (1, 1, 0.9))},
                           "logistic4": {"initial_guess": [1.0, 1.0, 1.0, 0.0], 
                                        "bounds": ((0, 0.1, 0, 0), (1.5, 50, 1, 0.9))},
                            "ll4": {"initial_guess": [0.4, 1.0, 1.0, 0.0], 
                                    "bounds": ((0.1,0,0,0), (1, 1, 50, 1))},
                            "ll4_R": {"initial_guess": [-5, 1.0, 5.0, 0], 
                                 "bounds":((-10,0,0,0), (0, 1.0, 50, 1))},
                            "logLogistR": {"initial_guess": [0.4, -4, 0], 
                                          "bounds":((0,-30,0), (1,-2,1))}}

    if default_param:
        parameters_guess = default_param_model[fitting_function]["initial_guess"]
       
    else:
        pass
    
    for i in tqdm(range(len(indexes))):
        x = X[i, :]
        y = Y[i, :]
    
        
        functions = {"fsigmoid": fsigmoid, 
                 "sigmoid_Wang": sigmoid_Wang, 
                "sigmoid_4_param": sigmoid_4_param,
                 "sigmoid_3_param": sigmoid_3_param, 
                 "logistic4": logistic4,  
                 "ll4": ll4, 
                 "ll4_R": ll4_R,
                 "logLogistR" : logLogistR}
        fitting_function_object = functions[fitting_function]
        try:
            r2_scores[i], fitting_parameters[i] = FitCurve(fitting_function_object, x, y, 
                                                               parameters_guess = parameters_guess,
                                                            bounds = default_param_model[fitting_function]["bounds"])
        except:
            r2_scores[i] = 0
    return r2_scores, fitting_parameters


def ShowResponseCurvesWithFitting(df, plots_in_row, plots_in_column, x_columns, y_columns, start_index=0, indexes=[],
                         fitting_function =None, fitting_parameters =None, pred_fitting_param = None):
    
    print("Figures titles: Index_DRUG_ID_COSMIC_ID (COSMIC_ID is a cell line)")
    fig = plt.figure(figsize=(14, 11))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    n_plots= plots_in_row*plots_in_column
    
    if len(indexes) == 0:
        indexes = df.index[start_index : start_index+n_plots]
        
    X = df.loc[indexes, x_columns].values.astype(np.float32)
    Y = df.loc[indexes, y_columns].values.astype(np.float32)
        
    try:
        for i in range(len(indexes)):
            x = X[i, :]
            y = Y[i, :]
            ind = indexes[i]

                
            ax = fig.add_subplot(plots_in_row, plots_in_column, i+1)
            ax.scatter(x,y)
            ax.set_title(str(ind)+"_"+str(df.loc[ind, "DRUG_ID"])+"_"+str(df.loc[ind, "COSMIC_ID"]))
            ax.set_xlabel("Scaled dosage")
            ax.set_ylabel("Normalised response")

            if fitting_function:
                functions = {"fsigmoid": fsigmoid, "sigmoid_Wang": sigmoid_Wang, 
                             "sigmoid_4_param": sigmoid_4_param, "sigmoid_3_param": sigmoid_3_param, 
                             "logistic4": logistic4, "ll4": ll4, "ll4_R": ll4_R, "logLogistR": logLogistR}
                fitting_function_object = functions[fitting_function]
                
#                 from fitting_curves.py import fitting_function_object

                x2 = np.linspace(0, 1, 10)

                if type(fitting_parameters) == str:
                    fit_param = df.loc[ind, fitting_parameters]
                else:
                    fit_param = df.loc[ind, fitting_parameters].values

                y_fit = fitting_function_object(x, *fit_param)
                y2 = fitting_function_object(x2, *fit_param)
                r2 = r2_score(y, y_fit)
                ax.plot(x2, y2, label= "R^2 fit = %0.4f"% r2)
                ax.legend()
            if pred_fitting_param:

                x3 = np.linspace(0, 1, 10) 
                fit_param = df.loc[ind, pred_fitting_param]    
                y_fit3 = fitting_function_object(x, *fit_param)
                y3 = fitting_function_object(x3, *fit_param)
                r2_pred = r2_score(y, y_fit3)
                ax.plot(x3, y3, color="red", label= "R^2 pred = %0.4f"% r2_pred)
                ax.legend()
                
    except:
        print("ERROR: Number of indexes does not correspond to number of graphs to plot")


def compute_r2_score(df, x_columns, y_columns, fitting_parameters, fitting_function="sigmoid_4_param"):
    functions = {"fsigmoid": fsigmoid, "sigmoid_Wang": sigmoid_Wang, 
                             "sigmoid_4_param": sigmoid_4_param, "sigmoid_3_param": sigmoid_3_param, 
                             "logistic4": logistic4, "ll4": ll4, "ll4_R":ll4_R, "logLogistR": logLogistR}
    fitting_function_object = functions[fitting_function]
    r2_scores=np.zeros(len(df.index))
    for i in range(len(df.index)):
        x = df.loc[df.index[i], x_columns]
        y = df.loc[df.index[i], y_columns]
        if type(fitting_parameters) == str:
            fit_param = df.loc[df.index[i], fitting_parameters]
        else:
            fit_param = df.loc[df.index[i], fitting_parameters].values
        y_fit = fitting_function_object(x, *fit_param)
        r2_scores[i] = r2_score(y, y_fit)
    return r2_scores

## Fitting Data

In [4]:

fitting_functions = ["sigmoid_Wang", "fsigmoid", "sigmoid_4_param", "sigmoid_3_param", "ll4", "ll4_R", "logistic4", "logLogistR"]

# Initialize a dictionary to store the number of good fits for each function
good_fits_summary = {function: {"total_good_fits": 0, "dataset_good_fits": {}} for function in fitting_functions}

### sigmoid_Wang

In [8]:
# Get a list of all dataset files in the folder
dataset_files = [f for f in os.listdir(_FOLDER_2) if os.path.isfile(os.path.join(_FOLDER_2, f))]

# Define the columns for concentration and normalized response
conc_columns = ["fd_num_" + str(i) for i in range(10)]
response_norm = ['norm_cells_' + str(i) for i in range(10)]

# Specify the fitting function as a string
fitting_function_name = "sigmoid_Wang"

# Loop through each dataset file
for file in tqdm(dataset_files):
    df = pd.read_csv(os.path.join(_FOLDER_2, file))
    
    # Perform fitting
    r2, fit_param = FittingColumn(df, df.index, x_columns=conc_columns, y_columns=response_norm,
                                  fitting_function=fitting_function_name, default_param=True)
    
 
    # Store the R^2 values and fitting parameters in the dataframe
    df[fitting_function_name + "_r2"] = r2
    df[fitting_function_name] = list(fit_param)
    
    # Print the shape of the dataframe after fitting
    print(f"Dataset: {file}, Shape after fitting: {df.shape}")
    
    # Filter the dataframe to only include rows with R^2 > 0 and count R^2 > 0.9
    df_filtered = df[df[fitting_function_name + "_r2"] > 0]
    num_r2_above_09 = df_filtered[df_filtered[fitting_function_name + "_r2"] > 0.9].shape[0]

    # Update the summary dictionary
    good_fits_summary[fitting_function_name]["total_good_fits"] += num_r2_above_09
    good_fits_summary[fitting_function_name]["dataset_good_fits"][file] = num_r2_above_09

    print(f"  R^2 > 0: {df_filtered.shape[0]}")
    print(f"  R^2 > 0.9: {num_r2_above_09}\n")

  4%|▍         | 1/24 [00:08<03:18,  8.63s/it]

Dataset: filtered_drug_profiles_123.csv, Shape after fitting: (2776, 49)
  R^2 > 0: 2776
  R^2 > 0.9: 2472



  8%|▊         | 2/24 [00:19<03:43, 10.17s/it]

Dataset: filtering_scenario_1.1.csv, Shape after fitting: (3667, 48)
  R^2 > 0: 3667
  R^2 > 0.9: 3243



 12%|█▎        | 3/24 [00:34<04:12, 12.03s/it]

...

Dataset: filtering_scenario_3.7.csv, Shape after fitting: (5788, 48)
  R^2 > 0: 5788
  R^2 > 0.9: 4966



 96%|█████████▌| 23/24 [07:28<00:19, 19.57s/it]

Dataset: filtering_scenario_3.8.csv, Shape after fitting: (6171, 48)
  R^2 > 0: 6171
  R^2 > 0.9: 5169



100%|██████████| 24/24 [07:48<00:00, 19.51s/it]

Dataset: filtering_scenario_3.9.csv, Shape after fitting: (6534, 48)
  R^2 > 0: 6534
  R^2 > 0.9: 5346



### fsigmoid

In [15]:
# Get a list of all dataset files in the folder
dataset_files = [f for f in os.listdir(_FOLDER_2) if os.path.isfile(os.path.join(_FOLDER_2, f))]

# Define the columns for concentration and normalized response
conc_columns = ["fd_num_" + str(i) for i in range(10)]
response_norm = ['norm_cells_' + str(i) for i in range(10)]

# Specify the fitting function as a string
fitting_function_name = "fsigmoid"

# Loop through each dataset file
for file in tqdm(dataset_files):
    df = pd.read_csv(os.path.join(_FOLDER_2, file))
    
    # Perform fitting
    r2, fit_param = FittingColumn(df, df.index, x_columns=conc_columns, y_columns=response_norm,
                                  fitting_function=fitting_function_name, default_param=True)
    
    # Store the R^2 values and fitting parameters in the dataframe
    df[fitting_function_name + "_r2"] = r2
    df[fitting_function_name] = list(fit_param)
    
    # Print the shape of the dataframe after fitting
    print(f"Dataset: {file}, Shape after fitting: {df.shape}")
    
    # Filter the dataframe to only include rows with R^2 > 0 and count R^2 > 0.9
    df_filtered = df[df[fitting_function_name + "_r2"] > 0]
    num_r2_above_09 = df_filtered[df_filtered[fitting_function_name + "_r2"] > 0.9].shape[0]
    
    # Update the summary dictionary
    good_fits_summary[fitting_function_name]["total_good_fits"] += num_r2_above_09
    good_fits_summary[fitting_function_name]["dataset_good_fits"][file] = num_r2_above_09

    print(f"  R^2 > 0: {df_filtered.shape[0]}")
    print(f"  R^2 > 0.9: {num_r2_above_09}\n")

  4%|▍         | 1/24 [00:08<03:22,  8.82s/it]

Dataset: filtered_drug_profiles_123.csv, Shape after fitting: (2776, 49)
  R^2 > 0: 2776
  R^2 > 0.9: 2472



  8%|▊         | 2/24 [00:23<04:23, 11.99s/it]

Dataset: filtering_scenario_1.1.csv, Shape after fitting: (3667, 48)
  R^2 > 0: 3667
  R^2 > 0.9: 3243



 12%|█▎        | 3/24 [00:38<04:41, 13.39s/it]

...

Dataset: filtering_scenario_3.7.csv, Shape after fitting: (5788, 48)
  R^2 > 0: 5788
  R^2 > 0.9: 4966



 96%|█████████▌| 23/24 [07:26<00:17, 17.90s/it]

Dataset: filtering_scenario_3.8.csv, Shape after fitting: (6171, 48)
  R^2 > 0: 6171
  R^2 > 0.9: 5169



100%|██████████| 24/24 [07:45<00:00, 19.40s/it]

Dataset: filtering_scenario_3.9.csv, Shape after fitting: (6534, 48)
  R^2 > 0: 6534
  R^2 > 0.9: 5346



### sigmoid_4_param

In [5]:
# Get a list of all dataset files in the folder
dataset_files = [f for f in os.listdir(_FOLDER_2) if os.path.isfile(os.path.join(_FOLDER_2, f))]

# Define the columns for concentration and normalized response
conc_columns = ["fd_num_" + str(i) for i in range(10)]
response_norm = ['norm_cells_' + str(i) for i in range(10)]

# Specify the fitting function as a string
fitting_function_name = "sigmoid_4_param"

# Loop through each dataset file
for file in tqdm(dataset_files):
    df = pd.read_csv(os.path.join(_FOLDER_2, file))
    
    # Perform fitting
    r2, fit_param = FittingColumn(df, df.index, x_columns=conc_columns, y_columns=response_norm,
                                  fitting_function=fitting_function_name, default_param=True)
    
    # Store the R^2 values and fitting parameters in the dataframe
    df[fitting_function_name + "_r2"] = r2
    df[fitting_function_name] = list(fit_param)
    
    # Print the shape of the dataframe after fitting
    print(f"Dataset: {file}, Shape after fitting: {df.shape}")
    
    # Filter the dataframe to only include rows with R^2 > 0 and count R^2 > 0.9
    df_filtered = df[df[fitting_function_name + "_r2"] > 0]
    num_r2_above_09 = df_filtered[df_filtered[fitting_function_name + "_r2"] > 0.9].shape[0]

    # Update the summary dictionary
    good_fits_summary[fitting_function_name]["total_good_fits"] += num_r2_above_09
    good_fits_summary[fitting_function_name]["dataset_good_fits"][file] = num_r2_above_09


    print(f"  R^2 > 0: {df_filtered.shape[0]}")
    print(f"  R^2 > 0.9: {num_r2_above_09}\n")


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:10<04:04, 10.64s/it]

Dataset: filtered_drug_profiles_123.csv, Shape after fitting: (2776, 49)
  R^2 > 0: 2775
  R^2 > 0.9: 2735



  8%|▊         | 2/24 [00:25<04:44, 12.92s/it]

Dataset: filtering_scenario_1.1.csv, Shape after fitting: (3667, 48)
  R^2 > 0: 3666
  R^2 > 0.9: 3602



...

Dataset: filtering_scenario_3.7.csv, Shape after fitting: (5788, 48)
  R^2 > 0: 5787
  R^2 > 0.9: 5665



 96%|█████████▌| 23/24 [09:03<00:24, 24.62s/it]

Dataset: filtering_scenario_3.8.csv, Shape after fitting: (6171, 48)
  R^2 > 0: 6170
  R^2 > 0.9: 6021



100%|██████████| 24/24 [09:31<00:00, 23.80s/it]

Dataset: filtering_scenario_3.9.csv, Shape after fitting: (6534, 48)
  R^2 > 0: 6532
  R^2 > 0.9: 6356



### sigmoid_3_param

In [17]:
# Get a list of all dataset files in the folder
dataset_files = [f for f in os.listdir(_FOLDER_2) if os.path.isfile(os.path.join(_FOLDER_2, f))]

# Define the columns for concentration and normalized response
conc_columns = ["fd_num_" + str(i) for i in range(10)]
response_norm = ['norm_cells_' + str(i) for i in range(10)]

# Specify the fitting function as a string
fitting_function_name = "sigmoid_3_param"

# Loop through each dataset file
for file in tqdm(dataset_files):
    df = pd.read_csv(os.path.join(_FOLDER_2, file))
    
    # Perform fitting
    r2, fit_param = FittingColumn(df, df.index, x_columns=conc_columns, y_columns=response_norm,
                                  fitting_function=fitting_function_name, default_param=True)
    
    # Store the R^2 values and fitting parameters in the dataframe
    df[fitting_function_name + "_r2"] = r2
    df[fitting_function_name] = list(fit_param)
    
    # Print the shape of the dataframe after fitting
    print(f"Dataset: {file}, Shape after fitting: {df.shape}")
    
    # Filter the dataframe to only include rows with R^2 > 0 and count R^2 > 0.9
    df_filtered = df[df[fitting_function_name + "_r2"] > 0]
    num_r2_above_09 = df_filtered[df_filtered[fitting_function_name + "_r2"] > 0.9].shape[0]

    # Update the summary dictionary
    good_fits_summary[fitting_function_name]["total_good_fits"] += num_r2_above_09
    good_fits_summary[fitting_function_name]["dataset_good_fits"][file] = num_r2_above_09


    print(f"  R^2 > 0: {df_filtered.shape[0]}")
    print(f"  R^2 > 0.9: {num_r2_above_09}\n")

  4%|▍         | 1/24 [00:10<04:08, 10.80s/it]

Dataset: filtered_drug_profiles_123.csv, Shape after fitting: (2776, 49)
  R^2 > 0: 2776
  R^2 > 0.9: 2687



  8%|▊         | 2/24 [00:24<04:29, 12.23s/it]

Dataset: filtering_scenario_1.1.csv, Shape after fitting: (3667, 48)
  R^2 > 0: 3667
  R^2 > 0.9: 3533



 12%|█▎        | 3/24 [00:40<04:58, 14.21s/it]

...

Dataset: filtering_scenario_3.7.csv, Shape after fitting: (5788, 48)
  R^2 > 0: 5788
  R^2 > 0.9: 5525



 96%|█████████▌| 23/24 [07:48<00:21, 21.57s/it]

Dataset: filtering_scenario_3.8.csv, Shape after fitting: (6171, 48)
  R^2 > 0: 6171
  R^2 > 0.9: 5845



100%|██████████| 24/24 [08:12<00:00, 20.50s/it]

Dataset: filtering_scenario_3.9.csv, Shape after fitting: (6534, 48)
  R^2 > 0: 6534
  R^2 > 0.9: 6133



### ll4

In [18]:
# Get a list of all dataset files in the folder
dataset_files = [f for f in os.listdir(_FOLDER_2) if os.path.isfile(os.path.join(_FOLDER_2, f))]

# Define the columns for concentration and normalized response
conc_columns = ["fd_num_" + str(i) for i in range(10)]
response_norm = ['norm_cells_' + str(i) for i in range(10)]

# Specify the fitting function as a string
fitting_function_name = "ll4"

# Loop through each dataset file
for file in tqdm(dataset_files):
    df = pd.read_csv(os.path.join(_FOLDER_2, file))
    
    # Perform fitting
    r2, fit_param = FittingColumn(df, df.index, x_columns=conc_columns, y_columns=response_norm,
                                  fitting_function=fitting_function_name, default_param=True)
    
    # Store the R^2 values and fitting parameters in the dataframe
    df[fitting_function_name + "_r2"] = r2
    df[fitting_function_name] = list(fit_param)
    
    # Print the shape of the dataframe after fitting
    print(f"Dataset: {file}, Shape after fitting: {df.shape}")
    
    # Filter the dataframe to only include rows with R^2 > 0 and count R^2 > 0.9
    df_filtered = df[df[fitting_function_name + "_r2"] > 0]
    num_r2_above_09 = df_filtered[df_filtered[fitting_function_name + "_r2"] > 0.9].shape[0]

    # Update the summary dictionary
    good_fits_summary[fitting_function_name]["total_good_fits"] += num_r2_above_09
    good_fits_summary[fitting_function_name]["dataset_good_fits"][file] = num_r2_above_09


    print(f"  R^2 > 0: {df_filtered.shape[0]}")
    print(f"  R^2 > 0.9: {num_r2_above_09}\n")

  4%|▍         | 1/24 [00:14<05:25, 14.15s/it]

Dataset: filtered_drug_profiles_123.csv, Shape after fitting: (2776, 49)
  R^2 > 0: 2776
  R^2 > 0.9: 2728



  8%|▊         | 2/24 [00:32<06:10, 16.86s/it]

Dataset: filtering_scenario_1.1.csv, Shape after fitting: (3667, 48)
  R^2 > 0: 3667
  R^2 > 0.9: 3592



 12%|█▎        | 3/24 [00:55<06:51, 19.61s/it]

...

Dataset: filtering_scenario_3.7.csv, Shape after fitting: (5788, 48)
  R^2 > 0: 5788
  R^2 > 0.9: 5651



 96%|█████████▌| 23/24 [10:37<00:28, 28.99s/it]

Dataset: filtering_scenario_3.8.csv, Shape after fitting: (6171, 48)
  R^2 > 0: 6170
  R^2 > 0.9: 6002



100%|██████████| 24/24 [11:09<00:00, 27.89s/it]

Dataset: filtering_scenario_3.9.csv, Shape after fitting: (6534, 48)
  R^2 > 0: 6532
  R^2 > 0.9: 6336



### ll4_R

In [19]:
# Get a list of all dataset files in the folder
dataset_files = [f for f in os.listdir(_FOLDER_2) if os.path.isfile(os.path.join(_FOLDER_2, f))]

# Define the columns for concentration and normalized response
conc_columns = ["fd_num_" + str(i) for i in range(10)]
response_norm = ['norm_cells_' + str(i) for i in range(10)]

# Specify the fitting function as a string
fitting_function_name = "ll4_R"

# Loop through each dataset file
for file in tqdm(dataset_files):
    df = pd.read_csv(os.path.join(_FOLDER_2, file))
    
    # Perform fitting
    r2, fit_param = FittingColumn(df, df.index, x_columns=conc_columns, y_columns=response_norm,
                                  fitting_function=fitting_function_name, default_param=True)
    
    # Store the R^2 values and fitting parameters in the dataframe
    df[fitting_function_name + "_r2"] = r2
    df[fitting_function_name] = list(fit_param)
    
    # Print the shape of the dataframe after fitting
    print(f"Dataset: {file}, Shape after fitting: {df.shape}")
    
    # Filter the dataframe to only include rows with R^2 > 0 and count R^2 > 0.9
    df_filtered = df[df[fitting_function_name + "_r2"] > 0]
    num_r2_above_09 = df_filtered[df_filtered[fitting_function_name + "_r2"] > 0.9].shape[0]

    # Update the summary dictionary
    good_fits_summary[fitting_function_name]["total_good_fits"] += num_r2_above_09
    good_fits_summary[fitting_function_name]["dataset_good_fits"][file] = num_r2_above_09


    print(f"  R^2 > 0: {df_filtered.shape[0]}")
    print(f"  R^2 > 0.9: {num_r2_above_09}\n")

  4%|▍         | 1/24 [00:13<05:09, 13.45s/it]

Dataset: filtered_drug_profiles_123.csv, Shape after fitting: (2776, 49)
  R^2 > 0: 2776
  R^2 > 0.9: 2727



  8%|▊         | 2/24 [00:31<05:50, 15.92s/it]

Dataset: filtering_scenario_1.1.csv, Shape after fitting: (3667, 48)
  R^2 > 0: 3667
  R^2 > 0.9: 3591



 12%|█▎        | 3/24 [00:53<06:31, 18.65s/it]

...

Dataset: filtering_scenario_3.7.csv, Shape after fitting: (5788, 48)
  R^2 > 0: 5788
  R^2 > 0.9: 5648



 96%|█████████▌| 23/24 [10:18<00:28, 28.19s/it]

Dataset: filtering_scenario_3.8.csv, Shape after fitting: (6171, 48)
  R^2 > 0: 6171
  R^2 > 0.9: 5999



100%|██████████| 24/24 [10:50<00:00, 27.10s/it]

Dataset: filtering_scenario_3.9.csv, Shape after fitting: (6534, 48)
  R^2 > 0: 6534
  R^2 > 0.9: 6334



### logistic4

In [20]:
# Get a list of all dataset files in the folder
dataset_files = [f for f in os.listdir(_FOLDER_2) if os.path.isfile(os.path.join(_FOLDER_2, f))]

# Define the columns for concentration and normalized response
conc_columns = ["fd_num_" + str(i) for i in range(10)]
response_norm = ['norm_cells_' + str(i) for i in range(10)]

# Specify the fitting function as a string
fitting_function_name = "logistic4"

# Loop through each dataset file
for file in tqdm(dataset_files):
    df = pd.read_csv(os.path.join(_FOLDER_2, file))
    
    # Perform fitting
    r2, fit_param = FittingColumn(df, df.index, x_columns=conc_columns, y_columns=response_norm,
                                  fitting_function=fitting_function_name, default_param=True)
    
    # Store the R^2 values and fitting parameters in the dataframe
    df[fitting_function_name + "_r2"] = r2
    df[fitting_function_name] = list(fit_param)
    
    # Print the shape of the dataframe after fitting
    print(f"Dataset: {file}, Shape after fitting: {df.shape}")
    
    # Filter the dataframe to only include rows with R^2 > 0 and count R^2 > 0.9
    df_filtered = df[df[fitting_function_name + "_r2"] > 0]
    num_r2_above_09 = df_filtered[df_filtered[fitting_function_name + "_r2"] > 0.9].shape[0]

    

    print(f"  R^2 > 0: {df_filtered.shape[0]}")
    print(f"  R^2 > 0.9: {num_r2_above_09}\n")

  4%|▍         | 1/24 [00:12<04:48, 12.55s/it]

Dataset: filtered_drug_profiles_123.csv, Shape after fitting: (2776, 49)
  R^2 > 0: 2775
  R^2 > 0.9: 2727



  8%|▊         | 2/24 [00:28<05:25, 14.78s/it]

Dataset: filtering_scenario_1.1.csv, Shape after fitting: (3667, 48)
  R^2 > 0: 3666
  R^2 > 0.9: 3591



 12%|█▎        | 3/24 [00:48<05:58, 17.09s/it]

...

Dataset: filtering_scenario_3.7.csv, Shape after fitting: (5788, 48)
  R^2 > 0: 5786
  R^2 > 0.9: 5650



 96%|█████████▌| 23/24 [09:22<00:25, 25.94s/it]

Dataset: filtering_scenario_3.8.csv, Shape after fitting: (6171, 48)
  R^2 > 0: 6168
  R^2 > 0.9: 6001



100%|██████████| 24/24 [09:52<00:00, 24.69s/it]

Dataset: filtering_scenario_3.9.csv, Shape after fitting: (6534, 48)
  R^2 > 0: 6530
  R^2 > 0.9: 6335



### logLogistR

In [11]:
# Get a list of all dataset files in the folder
dataset_files = [f for f in os.listdir(_FOLDER_2) if os.path.isfile(os.path.join(_FOLDER_2, f))]

# Define the columns for concentration and normalized response
conc_columns = ["fd_num_" + str(i) for i in range(10)]
response_norm = ['norm_cells_' + str(i) for i in range(10)]

# Specify the fitting function as a string
fitting_function_name = "logLogistR"

# Loop through each dataset file
for file in tqdm(dataset_files):
    df = pd.read_csv(os.path.join(_FOLDER_2, file))
    
    # Perform fitting
    r2, fit_param = FittingColumn(df, df.index, x_columns=conc_columns, y_columns=response_norm,
                                  fitting_function=fitting_function_name, default_param=True)
    
    # Store the R^2 values and fitting parameters in the dataframe
    df[fitting_function_name + "_r2"] = r2
    df[fitting_function_name] = list(fit_param)
    
    # Print the shape of the dataframe after fitting
    print(f"Dataset: {file}, Shape after fitting: {df.shape}")
    
    # Filter the dataframe to only include rows with R^2 > 0 and count R^2 > 0.9
    df_filtered = df[df[fitting_function_name + "_r2"] > 0]
    num_r2_above_09 = df_filtered[df_filtered[fitting_function_name + "_r2"] > 0.9].shape[0]

    print(f"  R^2 > 0: {df_filtered.shape[0]}")
    print(f"  R^2 > 0.9: {num_r2_above_09}\n")

  4%|▍         | 1/24 [00:01<00:24,  1.07s/it]

Dataset: filtered_drug_profiles_123.csv, Shape after fitting: (2776, 49)
  R^2 > 0: 0
  R^2 > 0.9: 0



  8%|▊         | 2/24 [00:02<00:27,  1.27s/it]

Dataset: filtering_scenario_1.1.csv, Shape after fitting: (3667, 48)
  R^2 > 0: 0
  R^2 > 0.9: 0



 12%|█▎        | 3/24 [00:04<00:30,  1.47s/it]

...

Dataset: filtering_scenario_3.7.csv, Shape after fitting: (5788, 48)
  R^2 > 0: 0
  R^2 > 0.9: 0



 96%|█████████▌| 23/24 [00:48<00:02,  2.22s/it]

Dataset: filtering_scenario_3.8.csv, Shape after fitting: (6171, 48)
  R^2 > 0: 0
  R^2 > 0.9: 0



100%|██████████| 24/24 [00:50<00:00,  2.12s/it]

Dataset: filtering_scenario_3.9.csv, Shape after fitting: (6534, 48)
  R^2 > 0: 0
  R^2 > 0.9: 0



In [21]:
# After running all the fits, compare the total number of good fits
for function, summary in good_fits_summary.items():
    print(f"Fitting function: {function}, Total good fits: {summary['total_good_fits']}")

    # Find which dataset had the most good fits for this function
    best_dataset = max(summary["dataset_good_fits"], key=summary["dataset_good_fits"].get)
    best_fits = summary["dataset_good_fits"][best_dataset]
    print(f"  Best dataset: {best_dataset}, Good fits: {best_fits}\n")

Fitting function: sigmoid_Wang, Total good fits: 111687
  Best dataset: filtering_scenario_3.15.csv, Good fits: 7112

Fitting function: fsigmoid, Total good fits: 223374
  Best dataset: filtering_scenario_3.15.csv, Good fits: 7112

Fitting function: sigmoid_4_param, Total good fits: 128682
  Best dataset: filtering_scenario_3.15.csv, Good fits: 8474

Fitting function: sigmoid_3_param, Total good fits: 125082
  Best dataset: filtering_scenario_3.15.csv, Good fits: 8140

Fitting function: ll4, Total good fits: 128310
  Best dataset: filtering_scenario_3.15.csv, Good fits: 8447

Fitting function: ll4_R, Total good fits: 128259
  Best dataset: filtering_scenario_3.15.csv, Good fits: 8445

Fitting function: logistic4, Total good fits: 0


ValueError: max() arg is an empty sequence

## Saving Fitted Datasets

In [6]:
def SplitFitParam(df):
    """Column fitting_param is splitted into separate columns """
    conc_columns= ["fd_num_"+str(i) for i in range(10)]
    response_norm = ['norm_cells_'+str(i) for i in range(10)]
    param_columns = ["DRUG_ID", "COSMIC_ID", "MAX_CONC"] + conc_columns + response_norm
    for i in range(len(df['fitting_param'].values[0])):
        param_col = "param_"+str(i+1)
        param_columns.append(param_col)
        df[param_col] = df['fitting_param'].apply(lambda x: x[i])
    return df[param_columns]

def FittedData(df, x_columns, y_columns, fitting_function, parameters_guess=[], default_param = True):
    
    r2, fit_param = FittingColumn(df, df.index, x_columns, y_columns, fitting_function, default_param = True)
    df["fitting_r2"] = r2
    df["fitting_param"] = fit_param
    df= df[df["fitting_r2"]!=0]
    return df

In [7]:
fitting_function_name = "sigmoid_4_param"

# Define concentration and response columns
conc_columns = ["fd_num_" + str(i) for i in range(10)]
response_norm = ['norm_cells_' + str(i) for i in range(10)]

# Now loop over all files and apply the fitting
for file in tqdm(os.listdir(_FOLDER_2)):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(_FOLDER_2, file))
        
        # Apply fitting function
        df_fitted = FittedData(df, conc_columns, response_norm, fitting_function_name, default_param=True)
        
        # Split fitting_param into separate columns
        df_fitted = SplitFitParam(df_fitted)

        print(df_fitted[df_fitted.columns[-10:]].head())
        
        # Construct the new file name
        scenario_num = file.split("_")[2]  
        new_file_name = f"fit_filtered_scenario_{scenario_num}.csv"
        
        # Save the filtered dataframe
        df_fitted.to_csv(os.path.join(_FOLDER_3, new_file_name), index=False)

  4%|▍         | 1/24 [00:10<04:09, 10.86s/it]

   norm_cells_4  norm_cells_5  norm_cells_6  norm_cells_7  norm_cells_8  \
0      0.718099      0.469376      0.173641      0.103125      0.016716   
1      0.720981      0.622026      0.378269      0.457085      0.358507   
2      0.976929      0.645822      0.096651      0.033273      0.019794   
3      0.121750      0.089484      0.069739      0.058550      0.048249   
4      0.795708      0.612518      0.325654      0.238841      0.229492   

   norm_cells_9   param_1   param_2    param_3       param_4  
0      0.010709  0.537148  1.029230 -10.076644  1.809627e-16  
1      0.362443  0.449166  1.614213 -11.144873  3.683538e-01  
2      0.017003  0.576565  1.057679 -29.296109  2.404038e-02  
3      0.031395  0.280757  1.014233 -14.114493  5.207879e-02  
4      0.187305  0.529826  1.248430 -10.386378  1.808381e-01  


  8%|▊         | 2/24 [00:25<04:53, 13.35s/it]

   norm_cells_4  norm_cells_5  norm_cells_6  norm_cells_7  norm_cells_8  \
0      0.720981      0.622026      0.378269      0.457085      0.358507   
1      0.976929      0.645822      0.096651      0.033273      0.019794   
2      0.213701      0.333376      0.440845      0.433407      0.248843   
3      0.795708      0.612518      0.325654      0.238841      0.229492   
4      0.954021      0.397550      0.359142      0.383976      0.380092   

   norm_cells_9   param_1   param_2    param_3   param_4  
0      0.362443  0.449166  1.614213 -11.144873  0.368354  
1      0.017003  0.576565  1.057679 -29.296109  0.024040  
2      0.273228  0.179233  1.455040 -22.753634  0.331866  
3      0.187305  0.529826  1.248430 -10.386378  0.180838  
4      0.344509  0.488135  1.595464 -50.961464  0.366906  


 12%|█▎        | 3/24 [00:45<05:35, 15.97s/it]

...

   norm_cells_4  norm_cells_5  norm_cells_6  norm_cells_7  norm_cells_8  \
0      0.720981      0.622026      0.378269      0.457085      0.358507   
1      0.976929      0.645822      0.096651      0.033273      0.019794   
2      0.213701      0.333376      0.440845      0.433407      0.248843   
3      0.795708      0.612518      0.325654      0.238841      0.229492   
4      0.954021      0.397550      0.359142      0.383976      0.380092   

   norm_cells_9   param_1   param_2    param_3   param_4  
0      0.362443  0.449166  1.614213 -11.144873  0.368354  
1      0.017003  0.576565  1.057679 -29.296109  0.024040  
2      0.273228  0.179233  1.455040 -22.753634  0.331866  
3      0.187305  0.529826  1.248430 -10.386378  0.180838  
4      0.344509  0.488135  1.595464 -50.961464  0.366906  


 96%|█████████▌| 23/24 [09:06<00:25, 25.62s/it]

   norm_cells_4  norm_cells_5  norm_cells_6  norm_cells_7  norm_cells_8  \
0      0.720981      0.622026      0.378269      0.457085      0.358507   
1      0.976929      0.645822      0.096651      0.033273      0.019794   
2      0.213701      0.333376      0.440845      0.433407      0.248843   
3      0.795708      0.612518      0.325654      0.238841      0.229492   
4      0.954021      0.397550      0.359142      0.383976      0.380092   

   norm_cells_9   param_1   param_2    param_3   param_4  
0      0.362443  0.449166  1.614213 -11.144873  0.368354  
1      0.017003  0.576565  1.057679 -29.296109  0.024040  
2      0.273228  0.179233  1.455040 -22.753634  0.331866  
3      0.187305  0.529826  1.248430 -10.386378  0.180838  
4      0.344509  0.488135  1.595464 -50.961464  0.366906  


100%|██████████| 24/24 [09:36<00:00, 24.03s/it]

   norm_cells_4  norm_cells_5  norm_cells_6  norm_cells_7  norm_cells_8  \
0      0.763920      0.656923      0.693909      0.633029      0.568700   
1      0.720981      0.622026      0.378269      0.457085      0.358507   
2      0.976929      0.645822      0.096651      0.033273      0.019794   
3      0.213701      0.333376      0.440845      0.433407      0.248843   
4      0.795708      0.612518      0.325654      0.238841      0.229492   

   norm_cells_9   param_1   param_2    param_3   param_4  
0      0.530703  0.276299  1.326407  -2.934387  0.436581  
1      0.362443  0.449166  1.614213 -11.144873  0.368354  
2      0.017003  0.576565  1.057679 -29.296109  0.024040  
3      0.273228  0.179233  1.455040 -22.753634  0.331866  
4      0.187305  0.529826  1.248430 -10.386378  0.180838  
